## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Dikson,73.5069,80.5464,6.93,91,100,16.26,RU,2021-10-30 21:07:25,overcast clouds
1,1,Punta Arenas,-53.1500,-70.9167,53.11,40,59,7.45,CL,2021-10-30 21:00:37,broken clouds
2,2,Hambantota,6.1241,81.1185,78.51,84,79,3.11,LK,2021-10-30 21:07:26,broken clouds
3,3,Ixtapa,20.7000,-105.2000,85.23,81,43,5.99,MX,2021-10-30 21:07:26,scattered clouds
4,4,San Carlos De Bariloche,-41.1456,-71.3082,62.35,14,0,17.27,AR,2021-10-30 21:04:19,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
matching_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
matching_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
2,2,Hambantota,6.1241,81.1185,78.51,84,79,3.11,LK,2021-10-30 21:07:26,broken clouds
4,4,San Carlos De Bariloche,-41.1456,-71.3082,62.35,14,0,17.27,AR,2021-10-30 21:04:19,clear sky
7,7,Vaini,-21.2000,-175.2000,75.22,77,9,16.24,TO,2021-10-30 21:07:00,clear sky
10,10,Maningrida,-12.0667,134.2667,79.93,76,97,4.74,AU,2021-10-30 21:07:29,overcast clouds
13,13,Celestun,20.8667,-90.4000,79.29,53,22,10.47,MX,2021-10-30 21:07:30,few clouds
23,23,Bambous Virieux,-20.3428,57.7575,70.21,77,32,6.06,MU,2021-10-30 21:07:34,scattered clouds
24,24,Castro,-24.7911,-50.0119,64.35,90,82,10.18,BR,2021-10-30 21:07:34,broken clouds
25,25,Rikitea,-23.1203,-134.9692,75.02,85,73,21.88,PF,2021-10-30 21:00:33,light rain
26,26,Lithakia,37.7167,20.8333,63.59,63,11,7.76,GR,2021-10-30 21:07:35,few clouds
27,27,Inongo,-1.9500,18.2667,72.95,85,51,3.60,CD,2021-10-30 21:07:35,broken clouds


In [9]:
# 4a. Determine if there are any empty rows.
matching_cities_df.describe()

,City_ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
count,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000
mean,330.996528,5.243926,18.631968,71.937292,74.135417,61.013889,6.972014
std,192.053319,23.663454,83.463046,5.810825,20.211229,38.218059,5.034394
min,2.000000,-45.575200,-175.200000,60.060000,12.000000,0.000000,0.000000
25%,165.500000,-14.792900,-44.990750,67.982500,66.750000,20.000000,3.200000
50%,327.000000,6.128700,21.843800,73.185000,81.000000,75.000000,5.715000
75%,490.250000,26.503925,93.379175,76.960000,89.000000,98.000000,9.570000
max,689.000000,45.582600,175.515300,79.930000,99.000000,100.000000,24.110000


In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis=0, how="any", inplace=False)

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Hambantota,LK,78.51,broken clouds,6.1241,81.1185,
4,San Carlos De Bariloche,AR,62.35,clear sky,-41.1456,-71.3082,
7,Vaini,TO,75.22,clear sky,-21.2000,-175.2000,
10,Maningrida,AU,79.93,overcast clouds,-12.0667,134.2667,
13,Celestun,MX,79.29,few clouds,20.8667,-90.4000,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("No lodging found nearby")

No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby
No lodging found nearby


In [17]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Hambantota,LK,78.51,broken clouds,6.1241,81.1185,Bungalow 63
4,San Carlos De Bariloche,AR,62.35,clear sky,-41.1456,-71.3082,Hotel Patagonia
7,Vaini,TO,75.22,clear sky,-21.2000,-175.2000,Keleti Beach Resort
10,Maningrida,AU,79.93,overcast clouds,-12.0667,134.2667,Dhukurrdji Lodge
13,Celestun,MX,79.29,few clouds,20.8667,-90.4000,Posada Lilia


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))